In [110]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import statsmodels.api as sm

In [111]:
data = pd.read_csv('data2018_2020.csv', encoding='cp949')

In [112]:
data['모기 개체수'] = data['모기 개체수'].str.replace(',', '').astype(int)

In [113]:
features = [
    '평균기온', '최저기온','일강수량(mm)', '평균 풍속(m/s)',
     '최소 상대습도(%)','평균 상대습도(%)', '평균 현지기압(hPa)', '평균 해면기압(hPa)',
    '합계 일조시간(hr)', '평균 전운량(1/10)', '1일전 모기 개체수','2일전 모기 개체수','3일전 모기 개체수']

target = '모기 개체수'

In [114]:
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [115]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [116]:
X_train_scaled_ = sm.add_constant(X_train_scaled)
X_test_scaled_ = sm.add_constant(X_test_scaled)

In [117]:
model = sm.OLS(y_train, X_train_scaled_)
results = model.fit()

In [118]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                 모기 개체수   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     245.6
Date:                Tue, 28 May 2024   Prob (F-statistic):          9.71e-182
Time:                        23:27:04   Log-Likelihood:                -3089.1
No. Observations:                 415   AIC:                             6206.
Df Residuals:                     401   BIC:                             6263.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        655.0529    362.074      1.809      0.0

In [119]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

In [120]:
y_train_hat = model.predict(X_train_scaled)
y_test_hat = model.predict(X_test_scaled)

In [121]:
print('Performance for TRAIN--------')
print('Train MSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)))
print('Train RMSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)**0.5))
print('Train R^2 : {:.4f}'.format(r2_score(y_train, y_train_hat)))

print('Performance for TEST--------')
print('Test MSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)))
print('Test RMSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)**0.5))
print('Test R^2 : {:.4f}'.format(r2_score(y_test, y_test_hat)))

Performance for TRAIN--------
Train MSE : 170997.6891
Train RMSE : 413.5187
Train R^2 : 0.8884
Performance for TEST--------
Test MSE : 200473.9925
Test RMSE : 447.7432
Test R^2 : 0.8389


In [122]:
# 선택된 특성들의 평균 계산
feature_means = data[features].mean().values.reshape(1, -1)

feature_means_scaled = scaler.transform(feature_means)

y_pred = model.predict(feature_means_scaled)

print(f"Predicted Mosquito Count(2018-2020): {y_pred[0]:.2f}")
print("-----------------------------")

Predicted Mosquito Count(2018-2020): 2496.19
-----------------------------


c:\Users\songs\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


# 2018-2020 Ridge

In [123]:
from sklearn.linear_model import Ridge

In [124]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=42)

In [125]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [126]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

alphas = [0.01, 0.1, 1, 10, 100, 1000]
train_r2_scores = []
val_r2_scores = []

for alpha in alphas:
    # Ridge 회귀 모델 생성 및 학습
    model = Ridge(alpha=alpha)
    model.fit(X_train_scaled, y_train)

    # 훈련 세트와 검증 세트에서의 예측 수행
    y_train_hat = model.predict(X_train_scaled)  
    y_val_hat = model.predict(X_val_scaled)  

    # R^2 값 계산
    train_r2 = r2_score(y_train, y_train_hat)
    val_r2 = r2_score(y_val, y_val_hat)
    
    # 결과 저장
    train_r2_scores.append(train_r2)
    val_r2_scores.append(val_r2)
    
    # 결과 출력
    print(f"Alpha: {alpha}")
    print(f"Train R^2: {train_r2:.4f}")
    print(f"Validation R^2: {val_r2:.4f}")
    print('---------------------')


Alpha: 0.01
Train R^2: 0.8825
Validation R^2: 0.8413
---------------------
Alpha: 0.1
Train R^2: 0.8823
Validation R^2: 0.8428
---------------------
Alpha: 1
Train R^2: 0.8763
Validation R^2: 0.8450
---------------------
Alpha: 10
Train R^2: 0.8198
Validation R^2: 0.7838
---------------------
Alpha: 100
Train R^2: 0.4516
Validation R^2: 0.4194
---------------------
Alpha: 1000
Train R^2: 0.0799
Validation R^2: 0.0706
---------------------


In [127]:
model = Ridge(alpha=1) #1 채택
model.fit(X_train_scaled, y_train)

Ridge(alpha=1)

In [128]:
y_train_hat = model.predict(X_train_scaled)
y_test_hat = model.predict(X_test_scaled)

In [129]:
coefficients = pd.DataFrame({
    'Feature': features,
    'Ridge Coefficient': model.coef_
})
print(coefficients)

         Feature  Ridge Coefficient
0           평균기온         253.809202
1           최저기온         282.528875
2       일강수량(mm)        -357.569453
3     평균 풍속(m/s)        -295.985602
4     최소 상대습도(%)        -427.561765
5     평균 상대습도(%)          84.588826
6   평균 현지기압(hPa)        -172.311036
7   평균 해면기압(hPa)        -192.288806
8    합계 일조시간(hr)         -24.481280
9   평균 전운량(1/10)         239.500421
10    1일전 모기 개체수        2915.859794
11    2일전 모기 개체수        1512.193478
12    3일전 모기 개체수         682.571744


In [130]:
print('Performance for TRAIN--------')
print('Train MSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)))
print('Train RMSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)**0.5))
print('Train R^2 : {:.4f}'.format(r2_score(y_train, y_train_hat)))

print('Performance for TEST--------')
print('Test MSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)))
print('Test RMSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)**0.5))
print('Test R^2 : {:.4f}'.format(r2_score(y_test, y_test_hat)))

Performance for TRAIN--------
Train MSE : 169835.8392
Train RMSE : 412.1114
Train R^2 : 0.8763
Performance for TEST--------
Test MSE : 175068.1925
Test RMSE : 418.4115
Test R^2 : 0.8875


# 2018-2020 MinMax Scale, Lasso

In [131]:
from sklearn.linear_model import Lasso

In [132]:
alphas = [0.01, 0.1, 1, 10, 100]
train_r2_scores = []
val_r2_scores = []

for alpha in alphas:
    # Ridge 회귀 모델 생성 및 학습
    model = Lasso(alpha=alpha)
    model.fit(X_train_scaled, y_train)

    # 훈련 세트와 검증 세트에서의 예측 수행
    y_train_hat = model.predict(X_train_scaled)  
    y_val_hat = model.predict(X_val_scaled)  

    # R^2 값 계산
    train_r2 = r2_score(y_train, y_train_hat)
    val_r2 = r2_score(y_val, y_val_hat)
    
    # 결과 저장
    train_r2_scores.append(train_r2)
    val_r2_scores.append(val_r2)
    
    # 결과 출력
    print(f"Alpha: {alpha}")
    print(f"Train R^2: {train_r2:.4f}")
    print(f"Validation R^2: {val_r2:.4f}")
    print('---------------------')

Alpha: 0.01
Train R^2: 0.8825
Validation R^2: 0.8412
---------------------
Alpha: 0.1
Train R^2: 0.8824
Validation R^2: 0.8421
---------------------
Alpha: 1
Train R^2: 0.8819
Validation R^2: 0.8485
---------------------
Alpha: 10
Train R^2: 0.8716
Validation R^2: 0.8576
---------------------
Alpha: 100
Train R^2: 0.6686
Validation R^2: 0.6704
---------------------


c:\Users\songs\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.312e+07, tolerance: 4.269e+04
  model = cd_fast.enet_coordinate_descent(


In [133]:
model = Lasso(alpha=1) #1채택
model.fit(X_train_scaled, y_train)

Lasso(alpha=1)

In [134]:
y_train_hat = model.predict(X_train_scaled)
y_test_hat = model.predict(X_test_scaled)

In [135]:
coefficients = pd.DataFrame({
    'Feature': features,
    'Lasso Coefficient': model.coef_
})
print(coefficients)

         Feature  Lasso Coefficient
0           평균기온         144.150579
1           최저기온         271.092125
2       일강수량(mm)        -370.851447
3     평균 풍속(m/s)        -312.589293
4     최소 상대습도(%)        -322.630551
5     평균 상대습도(%)           0.000000
6   평균 현지기압(hPa)          -0.000000
7   평균 해면기압(hPa)        -218.630218
8    합계 일조시간(hr)          -0.000000
9   평균 전운량(1/10)         267.282621
10    1일전 모기 개체수        4035.261306
11    2일전 모기 개체수        1322.710177
12    3일전 모기 개체수           0.000000


In [136]:
print('Performance for TRAIN--------')
print('Train MSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)))
print('Train RMSE : {:.4f}'.format(mean_squared_error(y_train, y_train_hat)**0.5))
print('Train R^2 : {:.4f}'.format(r2_score(y_train, y_train_hat)))

print('Performance for TEST--------')
print('Test MSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)))
print('Test RMSE : {:.4f}'.format(mean_squared_error(y_test, y_test_hat)**0.5))
print('Test R^2 : {:.4f}'.format(r2_score(y_test, y_test_hat)))

Performance for TRAIN--------
Train MSE : 162117.3636
Train RMSE : 402.6380
Train R^2 : 0.8819
Performance for TEST--------
Test MSE : 180585.8355
Test RMSE : 424.9539
Test R^2 : 0.8840
